In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

# 设置设备（优先使用GPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n使用设备: {device}")

# 1. 加载预训练模型
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
model = nn.Sequential(*list(model.children())[:-1])  # 移除最后一层，保留主干网络
model = model.to(device).eval()  # 设置为评估模式
print(f"\n加载模型：ResNet-50 loaded. Output embedding dim: 2048")


使用设备: cpu

加载模型：ResNet-50 loaded. Output embedding dim: 2048


In [12]:
# 2. 定义图像预处理
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# 3. 创建自定义数据集
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, f) for f in os.listdir(root_dir) 
                          if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_path

# 4. 创建数据集和加载器
cat_dataset = ImageDataset("/mnt/workspace/ai/datasets/cat", transform=preprocess)
dog_dataset = ImageDataset("/mnt/workspace/ai/datasets/dog", transform=preprocess)

cat_loader = DataLoader(cat_dataset, batch_size=8, shuffle=False)
dog_loader = DataLoader(dog_dataset, batch_size=8, shuffle=False)

# 5. 特征提取函数
def extract_features(loader, model):
    features = []
    paths = []
    with torch.no_grad():  # 禁用梯度计算
        for images, batch_paths in loader:
            images = images.to(device)
            outputs = model(images)
            # 压缩空间维度 (batch, 2048, 1, 1) -> (batch, 2048)
            features.append(outputs.squeeze())
            paths.extend(batch_paths)
    return torch.cat(features), paths

# 6. 提取特征
cat_features, cat_paths = extract_features(cat_loader, model)
dog_features, dog_paths = extract_features(dog_loader, model)

# 7. 输出结果
print("\nCat embeddings:", cat_features.shape)
# for path, feat in zip(cat_paths, cat_features):
#     print(f"{os.path.basename(path)}: {feat.cpu().numpy()[:5]}... (shape: {feat.shape})")

print("\nDog embeddings:", dog_features.shape)
# for path, feat in zip(dog_paths, dog_features):
#     print(f"{os.path.basename(path)}: {feat.cpu().numpy()[:5]}... (shape: {feat.shape})")



Cat embeddings: torch.Size([20, 2048])

Dog embeddings: torch.Size([20, 2048])


In [13]:
import numpy as np
from sklearn.linear_model import LogisticRegression

total_features = np.vstack((cat_features , dog_features))
print("合并后特征：", total_features.shape)

labels = [1]*len(cat_features) + [0]*len(dog_features)
print("生成标签：", len(labels))

# --- 训练逻辑回归模型 ---
print("\n训练二分类逻辑回归模型...")
clf = LogisticRegression(max_iter=1000) 
X = total_features
y = labels
clf.fit(X, y)
print("训练完成！")

合并后特征： (40, 2048)
生成标签： 40

训练二分类逻辑回归模型...
训练完成！


In [16]:
import matplotlib.pyplot as plt
import glob
import random

print("\n预测函数:")
def predict_sentiment(image_path):
    image = Image.open(image_path).convert("RGB")
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)  # 增加批次维度
    
    # 4. 提取特征
    with torch.no_grad():
        features = model(input_batch)
        
    # 展平特征向量 [1, 2048, 1, 1] -> [1, 2048]
    features = torch.flatten(features, 1)  
    
    proba = clf.predict_proba(features)[0]
    prediction = clf.predict(features)[0]
    
    sentiment = "猫" if prediction == 1 else "狗"
    print(f"动物分类: {sentiment} (置信度: {max(proba):.2%}) -> {image_path}")

    # plt.imshow(image)
    # plt.axis('off')  # 关闭坐标轴
    # plt.show()


print("\n测试:")
def get_random_files_glob(folder_path, num_files=5):
    """使用glob实现的版本（支持文件模式过滤）"""
    all_files = glob.glob(os.path.join(folder_path, "*"))
    all_files = [f for f in all_files if f.lower().endswith(('.jpg', '.png'))]
    return random.sample(all_files, min(num_files, len(all_files)))
    
for review in get_random_files_glob("/mnt/workspace/ai/datasets/temp/validation/Cat/"):
    predict_sentiment(review)
for review in get_random_files_glob("/mnt/workspace/ai/datasets/temp/validation/Dog/"):
    predict_sentiment(review)


预测函数:

测试:
动物分类: 猫 (置信度: 99.26%) -> /mnt/workspace/ai/datasets/temp/validation/Cat/at-conf-0.45034844-t-1651311051196.jpg
动物分类: 狗 (置信度: 73.08%) -> /mnt/workspace/ai/datasets/temp/validation/Cat/ar-conf-0.55699575-t-1659113731851.jpg
动物分类: 猫 (置信度: 98.89%) -> /mnt/workspace/ai/datasets/temp/validation/Cat/ca-conf-0.3730776-t-1657754105074.jpg
动物分类: 猫 (置信度: 98.73%) -> /mnt/workspace/ai/datasets/temp/validation/Cat/br-conf-0.17578125-t-1645301235269.jpg
动物分类: 猫 (置信度: 95.16%) -> /mnt/workspace/ai/datasets/temp/validation/Cat/be-conf-0.21952455-t-1648458453791.jpg
动物分类: 狗 (置信度: 82.40%) -> /mnt/workspace/ai/datasets/temp/validation/Dog/cz-conf-0.60546875-t-1645212375080.jpg
动物分类: 狗 (置信度: 98.78%) -> /mnt/workspace/ai/datasets/temp/validation/Dog/ec-conf-0.13535422-t-1656356456266.jpg
动物分类: 狗 (置信度: 75.53%) -> /mnt/workspace/ai/datasets/temp/validation/Dog/ec-conf-0.50390625-t-1638406234410.jpg
动物分类: 狗 (置信度: 95.67%) -> /mnt/workspace/ai/datasets/temp/validation/Dog/de-conf-0.42777854-t-16477926